In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.colors import LogNorm

import h5py

In [ ]:
#change path to your location
pathToSimData="/Users/dnoonan/Work/AIinPixel/simulated ptychography dataset"

with h5py.File(f"{pathToSimData}/data_cell_phase_n2e7.h5", "r") as f:
    data = f['exchange/data'][...]
with h5py.File(f"{pathToSimData}/data_cell_phase_n2e7_ref.h5", "r") as f:
    data_ref = f['exchange/data'][...]
    
data = data[0,:,36-16:36+16,36-16:36+16].real**2
data_ref = data_ref[0,:,36-16:36+16,36-16:36+16].real**2

Setup simple autoencoder

examples can be found here: https://keras.io/examples/vision/autoencoder/

In [ ]:
from tensorflow.keras.layers import Input, Flatten, MaxPooling2D, Dense, Conv2D, Conv2DTranspose, Reshape, UpSampling2D

from tensorflow.keras.models import Model

In [ ]:
#input is a 32x32 image
inputs = Input(shape=(32,32,1),name='input')

#add a convolutional layer, 8 filters with 3x3 kernel size
x = Conv2D(filters=8,
           kernel_size=(3,3),   
           activation='relu',
           padding='same',
           name='convLayer'
          )(inputs)

x = Flatten(name='flattenLayer')(x)

encodedLayer = Dense(64,
                     activation='relu',
                     name='denseLayer')(x)

d = Dense(32*32*8,
          name='decoderDense')(encodedLayer)

d = Reshape((32,32,8))(d)

d = Conv2DTranspose(8,(3,3),activation="relu", padding="same")(d)

decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same',name='decoded')(d)

In [ ]:
autoencoder = Model(inputs,decoded)

In [ ]:
autoencoder.summary()

Preprocessing:
Normalize data

In [ ]:
dataNorm = data / data.sum(axis=(1,2)).reshape(-1,1,1)
dataNorm_ref = data_ref / data_ref.sum(axis=(1,2)).reshape(-1,1,1)

dataNorm.shape=(-1,32,32,1)
dataNorm_ref.shape=(-1,32,32,1)

In [ ]:
autoencoder.compile(optimizer='adam',loss='mse')

In [ ]:
history = autoencoder.fit(dataNorm_ref,dataNorm_ref,
                          epochs=2,
                          validation_split=0.1)

In [ ]:
dataNorm_Out = autoencoder.predict(dataNorm)

In [ ]:
plt.subplot
fig,ax = plt.subplots(4,2,figsize=(8,15))
#ax = ax.flatten()
#get 4 random images
N = np.random.choice(np.arange(dataNorm.shape[0]),4)
for i in range(4):
    im = ax[i][0].imshow(dataNorm[N[i]],norm=LogNorm())  
    fig.colorbar(im,ax=ax[i][0])
    im = ax[i][1].imshow(dataNorm_Out[N[i]],norm=LogNorm())
    fig.colorbar(im,ax=ax[i][1])
#     ax[i][1].colorbar()
